In [1]:
import requests
import urllib.request 
from urllib.request import urlopen 
from bs4 import BeautifulSoup
import re
import json
import pandas as pd
import numpy as np
from numpy.random import randint 
from datetime import date
import datetime as dt
import os 
import glob

In [4]:
rootpage = 'https://www.rottentomatoes.com/sitemap_' 
end = '.xml'
today = date.today()

def getLinks(pageUrl):
    global pages 
    for sitemap_page in range(0, 25): 
        pages = set() 
        html = urlopen(rootpage+str(sitemap_page)+end)
        bsObj = BeautifulSoup(html)
        
        for link in bsObj.findAll('loc'):
            replacers = ['trailers/', 'pictures/']
            link = str(link).replace('</loc>', '').split('.com')
            
            if link[1] not in pages: 
                pages.add(link[1])
    
        pages = pd.DataFrame(pages, columns=['links'])
        pages.to_csv(r'/Users/mimiyufanyou/Documents/GitHub/rotten_movies/data/movie_links/links_'+str(today)+'_'+str(sitemap_page)+'.csv', sep=',')
        
        print('saved to csv:','links_'+str(today)+'_'+str(sitemap_page)+'.csv', dt.datetime.now())

getLinks('')
        

saved to csv: links_2019-11-17_0.csv 2019-11-17 11:23:10.668879
saved to csv: links_2019-11-17_1.csv 2019-11-17 11:23:12.585492
saved to csv: links_2019-11-17_2.csv 2019-11-17 11:23:14.708517
saved to csv: links_2019-11-17_3.csv 2019-11-17 11:23:16.476187
saved to csv: links_2019-11-17_4.csv 2019-11-17 11:23:18.533097
saved to csv: links_2019-11-17_5.csv 2019-11-17 11:23:20.885766
saved to csv: links_2019-11-17_6.csv 2019-11-17 11:23:22.924453
saved to csv: links_2019-11-17_7.csv 2019-11-17 11:23:27.338525
saved to csv: links_2019-11-17_8.csv 2019-11-17 11:23:29.358984
saved to csv: links_2019-11-17_9.csv 2019-11-17 11:23:31.620002
saved to csv: links_2019-11-17_10.csv 2019-11-17 11:23:33.581253
saved to csv: links_2019-11-17_11.csv 2019-11-17 11:23:39.416555
saved to csv: links_2019-11-17_12.csv 2019-11-17 11:23:41.574564
saved to csv: links_2019-11-17_13.csv 2019-11-17 11:23:43.869375
saved to csv: links_2019-11-17_14.csv 2019-11-17 11:23:46.269565
saved to csv: links_2019-11-17_15.c

In [5]:
filepath = r'/Users/mimiyufanyou/Documents/GitHub/rotten_movies/data/movie_links/*.csv'
df_links = pd.concat([pd.read_csv(f) for f in glob.glob(filepath)], ignore_index = True)
df_links = list(df_links['links'])

In [6]:
trimmed_links = set()

for links in df_links:
    links = links.replace('/pictures/', '').replace('/trailers/', '') 
    trimmed_links.add(links)

df_links = pd.DataFrame(df_links)

In [7]:
baseurl = 'https://www.rottentomatoes.com'
listlength = len(trimmed_links)
movie_df = pd.DataFrame(columns=['url', 'title', 'tomatoScore', 'popcornScore'])
today = date.today()

def getScores(link):
    try: 
        siteurl = urlopen(baseurl+link)

        bsObj = BeautifulSoup(siteurl)
        movie_title = str(bsObj).split('"name":"')[1].split('"')[0]
        #print(movie_title)

        tomatoScore = str(bsObj).split('ratingValue":')[1].split(',"')[0]
        #print(tomatoScore)

        ems_id = str(bsObj).split('emsId":"')[1].split('"')[0]
        ems_req_url = f'https://www.rottentomatoes.com/napi/audienceScore/{ems_id}'

        ems_response = requests.get(ems_req_url)
        ems_soup = BeautifulSoup(ems_response.text, 'html.parser')
        ems_text_response = ems_response.text
        json_ems = json.loads(ems_text_response)
        popcornScore = json_ems['audienceScoreAll']['score']
        #print(popcornScore)

        movie_df.loc[link] = [link, movie_title, tomatoScore, popcornScore]
    except: 
        pass

In [12]:
for batch in range(50,100):
    for link in df_links.iloc[batch]: 
        movie_name = df_links.iloc[batch][0]
        getScores(movie_name)

movie_df.to_csv(r'/Users/mimiyufanyou/Documents/GitHub/rotten_movies/data/movie_scores/scores_'+str(today)+'_'+str(batch)+'.csv', sep=',')
print('saved to csv:','scores_'+str(today)+'_'+str(batch)+'.csv', dt.datetime.now())

saved to csv: scores_2019-11-17_99.csv 2019-11-17 11:28:24.829414


In [44]:
filepath = r'/Users/mimiyufanyou/Documents/GitHub/rotten_movies/data/movie_scores/*.csv'
df_scores = pd.concat([pd.read_csv(f) for f in glob.glob(filepath)], ignore_index = True)

df_scores = df_scores[['url', 'title', 'tomatoScore', 'popcornScore']]
df_scores['squidScore'] = df_scores['tomatoScore']- df_scores['popcornScore']

df_final = df_scores[['url', 'title', 'tomatoScore', 'popcornScore', 'squidScore']].dropna(how='any')
df_final.to_csv(r'/Users/mimiyufanyou/Documents/GitHub/rotten_movies/data/movie_final/final'+str(today)+'.csv', sep=',', index_label='id')

df_final.head()


,url,title,tomatoScore,popcornScore,squidScore
2,/m/mickey_blue_eyes,Mickey Blue Eyes,45.0,42,3.0
3,/m/clerks,Clerks,88.0,89,-1.0
4,/m/schlock,Schlock,67.0,42,25.0
7,/m/mickey_blue_eyes,Mickey Blue Eyes,45.0,42,3.0
10,/m/mickey_blue_eyes,Mickey Blue Eyes,45.0,42,3.0


In [41]:
filepath = r'/Users/mimiyufanyou/Documents/GitHub/rotten_movies/data/movie_final/*.csv'
df_final = pd.concat([pd.read_csv(f) for f in glob.glob(filepath)], ignore_index = True)